# Other examples
A notebook to test simple layers

In [1]:
%cd ../..

e:\KTorch


C:\Users\Karim\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# imports
import numpy as np
from nn import Flatten, Linear, Dropout
from core import KTorch
from autograd import Tensor

### 1. Flatten

In [3]:
# create a tensor
np.random.seed(0)
x = np.random.randn(6, 128, 128, 3).astype(np.float32)
x_tensor = Tensor(x)
flatten = Flatten()
flatten(x_tensor).shape, 128*128*3

((6, 49152), 49152)

In [4]:
# Compare output
flattened = flatten(x_tensor)
np_flattened = x.reshape(6, -1)
np.equal(flattened.data, np_flattened).all()

True

#### Test backpropagation

In [5]:
# Create input
np.random.seed(0)
x = np.random.randn(6, 128, 128, 3).astype(np.float32)
x_tensor = Tensor(x)

In [6]:
# Create model
flatten = Flatten()
linear = Linear(128*128*3, 10)

# Forward pass
flattened = flatten(x_tensor)
output = linear(flattened)
output.shape

(6, 10)

In [7]:
# Manual forward pass

# Parameters
w = linear.weight.data
b = linear.bias.data

np_flattened = x.reshape(6, -1)
manual_output = np_flattened @ w + b
np.equal(output.data, manual_output).all()

True

In [8]:
# Backward pass
output.backward()

In [9]:
# Manual backward pass
grad_output = np.ones_like(output.data)
grad_flattened = grad_output @ w.T
grad_flattened = grad_flattened.reshape(6, 128, 128, 3)
grad_x = grad_flattened
w_grad = np_flattened.T @ grad_output
b_grad = grad_output.sum(axis=0)

np.equal(x_tensor.grad.data, grad_x).all(), np.equal(linear.weight.grad.data, w_grad).all(), np.equal(linear.bias.grad.data, b_grad).all()

(True, True, True)

### 2. Dropout

In [10]:
# Create tensor
np.random.seed(0)
x = np.random.randn(32, 128).astype(np.float32)
x_tensor = Tensor(x)

In [16]:
# Dropout
dropout = Dropout(0.5)
linear = Linear(128, 10)

output, mask = dropout(x_tensor)
output = linear(output)

output.shape

(32, 10)

In [17]:
# Manual forward propagation
mask_np = mask.data
w = linear.weight.data
b = linear.bias.data


masked = x * mask_np * (1.0 / 0.5)
output_np = masked @ w + b

In [18]:
output_np.shape, np.equal(output.data, output_np).all()

((32, 10), True)

In [19]:
# Backward pass
output.backward()

In [29]:
# Manual backward pass
grad_output = np.ones_like(output.data)
grad_masked = grad_output @ w.T
grad_x = grad_masked * mask_np * (1.0 / 0.5)
w_grad = masked.T @ grad_output
b_grad = grad_output.sum(axis=0)

np.equal(x_tensor.grad.data, grad_x).all(), np.equal(linear.weight.grad, w_grad).all(), np.equal(linear.bias.grad, b_grad).all()

(True, True, True)